### Homework 23
04.02.2024
V. Ivanouskaya
v1

Решить задачу регрессии.
Получить максимальную метрику качества (R2, RMSE).

Введение
Целью данной задачи является прогнозирование суммы пятничных распродаж в магазинах сети Walmart с помощью построения регрессионных моделей и их анализа.
Набор данных состоит из исторических данных, которые охватывают продажи с 2010-02-05 по 2012-11-01, предствленных в файле Walmart.csv
#Использовать приплайны.
#Выполнить поиск лучших параметров
#Кроссвалидация.
#Сохранить и загрузить моделль. 

In [2]:
import pandas as pd
data = pd.read_csv('Walmart.csv')
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


######  предварительная обработка данных

In [13]:
# Преобразование колонки Date в формат даты
data['Date'] = pd.to_datetime(data['Date'], format='%d-%m-%Y')
data['Date'].info()
# Создание новых признаков: года и месяца
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data.head()

<class 'pandas.core.series.Series'>
RangeIndex: 6435 entries, 0 to 6434
Series name: Date
Non-Null Count  Dtype         
--------------  -----         
6435 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 50.4 KB


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3


In [14]:
# Проверка на наличие дубликатов
duplicates = data.duplicated().sum()
duplicates

0

###### Разделение данных

In [15]:
from sklearn.model_selection import train_test_split

# Определение признаков и целевой переменной
X = data.drop(['Weekly_Sales', 'Date'], axis=1)  # Удаление колонки Date, так как год и месяц уже включены
y = data['Weekly_Sales']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((5148, 8), (1287, 8))

###### Cоздание Pipeline

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# Настройка пайплайна с Ridge регрессией
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge())
])

# Определение сетки параметров для поиска
param_grid = {
    'regressor__alpha': [0.1, 1.0, 10.0]  # Параметр регуляризации
}

# Использование GridSearchCV для автоматизации поиска и кросс-валидации
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
pipeline

Pipeline(steps=[('scaler', StandardScaler()), ('regressor', Ridge())])

In [25]:
grid_search.fit(X_train, y_train) # Обучение

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('regressor', Ridge())]),
             n_jobs=-1, param_grid={'regressor__alpha': [0.1, 1.0, 10.0]},
             scoring='neg_mean_squared_error', verbose=2)

In [28]:
print("Лучшие параметры:", grid_search.best_params_)
best_model = grid_search.best_estimator_

Лучшие параметры: {'regressor__alpha': 10.0}


In [29]:
y_pred = best_model.predict(X_test)  
print("R2:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R2: 0.15632867217206625
RMSE: 521337.28939178295


###### Вывод:
1) коэффициент детерминации показывает, что  модель может объяснить только около 15.63% вариации целевой переменной. 
Это относительно низкое значение R2 показывает, что модель не очень хорошо подходит для данных или что данные сложно предсказать с использованием линейной зависимости.
2) RMSE говорит о том, что средняя ошибка прогноза модели составляет примерно 521,337 единиц продаж.Величина RMSE указывает на то, что прогнозы могут существенно отклоняться от фактических значений.

In [30]:
import joblib
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']